In [ ]:
from dotenv import load_dotenv
from phmlondon.snow_utils import SnowflakeConnection
import numpy as np

load_dotenv()
conn = SnowflakeConnection()
conn.use_database("INTELLIGENCE_DEV")
conn.use_schema("AI_CENTRE_DEV")

## (1) Exploring clustering of patient codes
(limit query to 1000 patients as keeps timing out)

In [ ]:
counts_by_patient = conn.session.sql("""
select o.patient_id, o.core_concept_id, count(o.core_concept_id) as code_count
from prod_dwh.analyst_primary_care.observation as o
group by o.patient_id, o.core_concept_id
order by code_count desc
""").to_pandas()

In [ ]:
total_num_codes = conn.session.sql("""
select o.core_concept_id, count(o.core_concept_id) as code_count
from prod_dwh.analyst_primary_care.observation as o
group by o.core_concept_id
order by code_count desc;
""").to_pandas()

In [ ]:
counts_by_patient

In [ ]:
len(counts_by_patient)

In [ ]:
total_num_codes

In [ ]:
# remove any codes where the number of that code in the whole dataset is less than 100
valid_codes = total_num_codes.loc[total_num_codes['CODE_COUNT'] >= 100]
valid_codes

In [ ]:
counts_by_patient_filtered = counts_by_patient[counts_by_patient['CORE_CONCEPT_ID'].isin(valid_codes['CORE_CONCEPT_ID'])]
counts_by_patient_filtered

In [ ]:
len(counts_by_patient_filtered)

In [ ]:
# don't actually know if the above is working

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(counts_by_patient_filtered['CODE_COUNT'], bins=100)
plt.show

In [ ]:
# Use log normalisation due to distribution

In [ ]:
import numpy as np

In [ ]:
counts_by_patient_filtered['LOG_CODE_COUNT'] = counts_by_patient_filtered['CODE_COUNT'].apply(lambda x: np.log(x))

In [ ]:
counts_by_patient_filtered.head()

In [ ]:
plt.hist(counts_by_patient_filtered['LOG_CODE_COUNT'], bins=100)
plt.show

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph

In [ ]:
counts_by_patient_filtered

In [ ]:
reshaped_data = counts_by_patient_filtered.pivot_table(index='PATIENT_ID', columns='CORE_CONCEPT_ID', values='LOG_CODE_COUNT', fill_value=0)
reshaped_data

In [ ]:
per = 200
it = 1000
metric = 'cosine'
pca = PCA(n_components=50)
data_pca_sample = pca.fit_transform(reshaped_data)
tsne_sample = TSNE(n_components=2, n_jobs=20, perplexity=per, n_iter=it, metric=metric, init ='pca', learning_rate='auto',).fit_transform(data_pca_sample)


In [ ]:
n_clusters = 10

In [ ]:
knn_graph = kneighbors_graph(tsne_sample, 100, include_self=False)

ward = AgglomerativeClustering(
    n_clusters=n_clusters, linkage="ward", connectivity=knn_graph
)

ward.fit(tsne_sample)

In [ ]:
fig, axs = plt.subplots()
axs.scatter(x=tsne_sample[:,0], y=tsne_sample[:,1])
bbox_props = dict(boxstyle="round", fc="w", ec="0.5", alpha=0.9)
axs.set_title(f"Clusters = {n_clusters}")
fig.set_size_inches(10, 10)

In [ ]:
fig, axs = plt.subplots()
axs.scatter(x=tsne_sample[:,0], y=tsne_sample[:,1], c=ward.labels_, cmap=plt.cm.nipy_spectral)
bbox_props = dict(boxstyle="round", fc="w", ec="0.5", alpha=0.9)

for x in range(n_clusters):
    w = [c == x for c in ward.labels_]
    cl = tsne_sample[w,:]
    cen = cl.mean(axis=0)
    #ax.annotate(x, cen)
    axs.text(cen[0], cen[1],str(x), ha="center", va="center", size=20, bbox=bbox_props)
axs.set_title(f"Clusters = {n_clusters}")
fig.set_size_inches(10, 10)

In [ ]:
import pandas as pd

In [ ]:
# find out what the clusters mean
labels = ward.labels_
print(labels.shape)
reshaped_data_reset = reshaped_data.reset_index()
clustered_data = pd.DataFrame({'patient_id': reshaped_data_reset['PATIENT_ID'], "cluster": labels})
clustered_data

In [ ]:
reshaped_data.head()

In [ ]:
most_popular_code = {patient: reshaped_data.loc[patient].idxmax() for patient in reshaped_data.index}

In [ ]:
most_popular_code

In [ ]:
clustered_data['most_popular_code'] = clustered_data['patient_id'].map(most_popular_code)

In [ ]:
clustered_data

In [ ]:
most_popular_code_for_sql = ",".join(map(str,clustered_data['most_popular_code']))
print(most_popular_code_for_sql)



In [ ]:
query = f"""
    SELECT DBID, name 
    FROM prod_dwh.analyst_primary_care.concept 
    WHERE DBID in ({most_popular_code_for_sql})
    """

most_popular_code_names = conn.session.sql(query).to_pandas()
print(most_popular_code_names)

In [ ]:
clustered_data.head()

In [ ]:
names = [most_popular_code_names.loc[most_popular_code_names['DBID'] == code]['NAME'] 
         for code in clustered_data['most_popular_code']]

In [ ]:
clustered_data['most_popular_code_name'] = names

In [ ]:
for cluster in range(n_clusters):
    print(f"Cluster {cluster}")
    print(clustered_data.loc[clustered_data['cluster'] == cluster, 'most_popular_code_name'])

#  clustered_data[clustered_data['cluster'] == 9]

(2) Now with larger dataset
Try filtering by disease codes only rather than all appointment codes
Limit data to only the last 10 years

In [ ]:
counts_by_patient = conn.session.sql("""
select o.patient_id, o.core_concept_id, count(o.core_concept_id) as code_count, c.name
from prod_dwh.analyst_primary_care.observation as o
join prod_dwh.analyst_primary_care.concept as c
on o.core_concept_id = c.dbid
where c.name like '%(disorder)%' 
and o.clinical_effective_date >= DATEADD(YEAR, -10, CURRENT_DATE)
group by o.patient_id, o.core_concept_id, c.name
order by code_count desc;
""").to_pandas()

In [ ]:
# Have skipped removing the rare codes this time

# Log normalise
counts_by_patient['LOG_CODE_COUNT'] = counts_by_patient['CODE_COUNT'].apply(lambda x: np.log(x))

In [ ]:
reshaped_data = counts_by_patient.pivot_table(index='PATIENT_ID', columns='CORE_CONCEPT_ID', values='LOG_CODE_COUNT', fill_value=0)